In [1]:
import sys

root_dir = '../../'
if root_dir not in sys.path:
    sys.path.append(root_dir)

import torch
from torch import nn, optim
import numpy as np
from sklearn.decomposition import TruncatedSVD
from scipy.sparse import csr_matrix

from modules import losses, models, samplers, regularizers, evaluators, trainers, datasets, distributions

In [2]:
# get data
dataset = datasets.ML100k()
n_user = dataset.n_user
n_item = dataset.n_item
train_set, test_set = dataset.get_train_and_test_set()

# device setting
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
train_set = torch.LongTensor(train_set).to(device)
test_set = torch.LongTensor(test_set).to(device)
#item_feature_set = torch.LongTensor(item_feature_set).to(device)

# set hyperparameters
lr = 1e-3
n_dim = 10

# set model
#distribution = distributions.Empirical()
#distribution = distributions.Gaussian()
#distribution = distributions.Gamma()
model = models.CollaborativeMetricLearning(n_user, n_item, n_dim).to(device)
#model = models.MutualProximityCML(n_user, n_item, distribution, n_dim).to(device)
optimizer = optim.Adam(model.parameters(), lr=lr)
criterion = losses.SumTripletLoss(margin=1).to(device)
sampler = samplers.BaseSampler(train_set, n_user, n_item, device=device, strict_negative=False)

# set evaluator
ks=[10, 50]
#evaluator = evaluators.RecallEvaluator(test_set, ks)
#evaluator = evaluators.UnpopularityEvaluator(test_set, ks)
evaluator = evaluators.F1ScoreEvaluator(test_set, ks)
#evaluator = evaluators.CoverageEvaluator(test_set, ks)
#evaluator = evaluators.HubnessEvaluator(test_set, ks)
trainer = trainers.BaseTrainer(model, optimizer, criterion, sampler)

In [3]:
trainer.fit(n_batch=256, n_epoch=10, valid_evaluator = evaluator, valid_per_epoch=10)

100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 940/940 [00:05<00:00, 174.60it/s]


In [4]:
trainer.valid_scores

,F1-score@10,F1-score@50,epoch,losses
0,0.093552,0.515629,0,NaN
0,0.057334,0.475971,10,0.470205


In [ ]:
import torch
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(device)